payment의 경우 하나로 쓰는게 의미가 없어보여서 다른 파일과 합쳐서 해보는 편이 더 좋을 것 같음. 

# 1. pledge-payment-lable 결정트리

이렇게 3개의 데이터를 쓸 경우

 - 기존의 두 파일을 합치는 경우 혈맹데이터(pledge)와 생존데이터(label)에 겹치는 인원이 33854명(혈맹에 들어있지 않은 유저도 존재하기 때문에 차이 발생)인 상황
 - 거기에 추가로 결제데이터(payment)가 합쳐지면서 결제내역이 없는 무과금유저들은 제거되어 해당 모델에서는 22282명의 인원만 남은 상태로 모델을 실행하게됨
 
-----> 40000명 -> 33854명 -> 22282명

In [13]:
# https://github.com/Swalloow/Kaggle/blob/master/Titanic%20Survivors/Titanic%20RandomForest.ipynb/
# 설명 : https://swalloow.github.io/decison-randomforest
import numpy as np
import pandas as pd


## Load Data


label_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_label.csv')
pledge_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_pledge.csv')
payment_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_payment.csv')


pledge_raw.dtypes

pledge = pledge_raw
label = label_raw
payment = payment_raw
pledge_raw.shape

pledge_grp = pledge.groupby('acc_id',as_index=False).sum()
payment_grp = payment.groupby('acc_id', as_index=False).sum()
pledge_label_grp = pd.merge(pledge_grp, 
                             label[['acc_id','survival_time']] ,
                            on ='acc_id', how='inner')
print(combat_label_grp.shape)

pledge_label_grp = pd.merge( pledge_label_grp, payment_grp[['acc_id','amount_spent']],
                            on = 'acc_id', how='inner')
pledge_grp.shape, pledge_label_grp.shape

# pledge_label = label.merge(pledge, on ='acc_id', how='inner')

# pledge_label_grp = pledge_label.groupby('acc_id',as_index=False).mean()

print(pledge_label_grp.shape)


pledge_label_grp['amount_spent'] = pledge_label_grp['amount_spent'].fillna(0)
pledge_label_grp.head()

data = pledge_label_grp.drop(['day','char_id'], axis =1)
data['survived'] = np.where(data['survival_time'] == 64, 1,0)
# 'survived','death'

data.drop(['acc_id','survival_time'], axis=1, inplace = True)

data.head()
# pd.isnull(pledge_label_grp).sum()

# data.survived.sum()

#훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split

y = data['survived']
x = data.iloc[:,:-1]





train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

## DecisionTree

from sklearn.tree import DecisionTreeClassifier

decision = DecisionTreeClassifier(max_depth = 4).fit(train_x,train_y)
# print(decision.score(train_x, train_y))
print(decision.score(test_x, test_y))

(23726, 14)
(22282, 16)
0.6142109199700823


In [14]:
from sklearn.tree import export_graphviz
import graphviz
import pydot

export_graphviz(decision,
                feature_names = x.columns,
                class_names = ['Death', 'Survived'],
                out_file = 'decisionTree1.dot',
                impurity = False,
                filled = True)

# Encoding 중요
# (graph,) = pydot.graph_from_dot_file('decisionTree1.dot', encoding='utf8')

# Dot 파일을 Png 이미지로 저장
# graph.write_png('decisionTree1.png')

with open('decisionTree1.dot') as f:
    dot_graph = str(open("decisionTree1.dot", "rb").read(), 'utf8')

src = graphviz.Source(dot_graph)
src.render('pledge-lable.gv', view=True)

'pledge-lable.gv.pdf'

# 2. activity-payment-lable 결정트리

이렇게 3개의 데이터를 쓸 경우

 - 기존의 두 파일을 합치는 경우에는 40000명의 유저가 모두 포함되는 상황이지만
 - 추가로 결제데이터(payment)가 합쳐지면서 결제내역이 없는 무과금유저들은 제거되어 해당 모델에서는 23726명의 인원만 남은 상태로 모델을 실행하게됨

In [15]:
# https://github.com/Swalloow/Kaggle/blob/master/Titanic%20Survivors/Titanic%20RandomForest.ipynb/
# 설명 : https://swalloow.github.io/decison-randomforest
import numpy as np
import pandas as pd


## Load Data


label_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_label.csv')
activity_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_activity.csv')
payment_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_payment.csv')


activity_raw.dtypes

activity = activity_raw
label = label_raw
payment = payment_raw
activity_raw.shape

activity_grp = activity.groupby('acc_id',as_index=False).sum()
payment_grp = payment.groupby('acc_id', as_index=False).sum()
activity_label_grp = pd.merge(activity_grp, 
                             label[['acc_id','survival_time']] ,
                            on ='acc_id', how='inner')
print(combat_label_grp.shape)

activity_label_grp = pd.merge( activity_label_grp, payment_grp[['acc_id','amount_spent']],
                            on = 'acc_id', how='inner')
activity_grp.shape, activity_label_grp.shape

# activity_label = label.merge(activity, on ='acc_id', how='inner')

# activity_label_grp = activity_label.groupby('acc_id',as_index=False).mean()

print(activity_label_grp.shape)


activity_label_grp['amount_spent'] = activity_label_grp['amount_spent'].fillna(0)
activity_label_grp.head()

data = activity_label_grp.drop(['day','char_id'], axis =1)
data['survived'] = np.where(data['survival_time'] == 64, 1,0)
# 'survived','death'

data.drop(['acc_id','survival_time'], axis=1, inplace = True)

data.head()
# pd.isnull(activity_label_grp).sum()

# data.survived.sum()

#훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split

y = data['survived']
x = data.iloc[:,:-1]





train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

## DecisionTree

from sklearn.tree import DecisionTreeClassifier

decision = DecisionTreeClassifier(max_depth = 6).fit(train_x,train_y)
# print(decision.score(train_x, train_y))
print(decision.score(test_x, test_y))

from sklearn.tree import export_graphviz
import graphviz
import pydot

export_graphviz(decision,
                feature_names = x.columns,
                class_names = ['Death', 'Survived'],
                out_file = 'decisionTree1.dot',
                impurity = False,
                filled = True)

# Encoding 중요
# (graph,) = pydot.graph_from_dot_file('decisionTree1.dot', encoding='utf8')

# Dot 파일을 Png 이미지로 저장
# graph.write_png('decisionTree1.png')

with open('decisionTree1.dot') as f:
    dot_graph = str(open("decisionTree1.dot", "rb").read(), 'utf8')

src = graphviz.Source(dot_graph)
src.render('activity-lable.gv', view=True)

(23726, 14)
(23726, 18)
0.6993537510536667


'activity-lable.gv.pdf'

# 3 combat-payment-lable 결정트리

이렇게 3개의 데이터를 쓸 경우

 - 기존의 두 파일을 합치는 경우에는 40000명의 유저가 모두 포함되는 상황이지만
 - 추가로 결제데이터(payment)가 합쳐지면서 결제내역이 없는 무과금유저들은 제거되어 해당 모델에서는 23726명의 인원만 남은 상태로 모델을 실행하게됨

In [16]:
# https://github.com/Swalloow/Kaggle/blob/master/Titanic%20Survivors/Titanic%20RandomForest.ipynb/
# 설명 : https://swalloow.github.io/decison-randomforest
import numpy as np
import pandas as pd


## Load Data


label_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_label.csv')
combat_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_combat.csv')
payment_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_payment.csv')


combat_raw.dtypes

combat = combat_raw
label = label_raw
payment = payment_raw
combat_raw.shape

combat_grp = combat.groupby('acc_id',as_index=False).sum()
payment_grp = payment.groupby('acc_id', as_index=False).sum()
combat_label_grp = pd.merge(combat_grp, 
                             label[['acc_id','survival_time']] ,
                            on ='acc_id', how='inner')
print(combat_label_grp.shape)

combat_label_grp = pd.merge( combat_label_grp, payment_grp[['acc_id','amount_spent']],
                            on = 'acc_id', how='inner')
combat_grp.shape, combat_label_grp.shape

# combat_label = label.merge(combat, on ='acc_id', how='inner')

# combat_label_grp = combat_label.groupby('acc_id',as_index=False).mean()

print(combat_label_grp.shape)


combat_label_grp['amount_spent'] = combat_label_grp['amount_spent'].fillna(0)
combat_label_grp.head()

data = combat_label_grp.drop(['day','char_id'], axis =1)
data['survived'] = np.where(data['survival_time'] == 64, 1,0)
# 'survived','death'

data.drop(['acc_id','survival_time'], axis=1, inplace = True)

data.head()
# pd.isnull(combat_label_grp).sum()

# data.survived.sum()

#훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split

y = data['survived']
x = data.iloc[:,:-1]





train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

## DecisionTree

from sklearn.tree import DecisionTreeClassifier

decision = DecisionTreeClassifier(max_depth = 6).fit(train_x,train_y)
# print(decision.score(train_x, train_y))
print(decision.score(test_x, test_y))

from sklearn.tree import export_graphviz
import graphviz
import pydot

export_graphviz(decision,
                feature_names = x.columns,
                class_names = ['Death', 'Survived'],
                out_file = 'decisionTree1.dot',
                impurity = False,
                filled = True)

# Encoding 중요
# (graph,) = pydot.graph_from_dot_file('decisionTree1.dot', encoding='utf8')

# Dot 파일을 Png 이미지로 저장
# graph.write_png('decisionTree1.png')

with open('decisionTree1.dot') as f:
    dot_graph = str(open("decisionTree1.dot", "rb").read(), 'utf8')

src = graphviz.Source(dot_graph)
src.render('combat-lable.gv', view=True)

(40000, 13)
(23726, 14)
0.6746277044113516


'combat-lable.gv.pdf'